In [ ]:
import os
os.environ['WANDB_DISABLED']="true"

# Install and load libraries

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib rouge_score evaluate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!nvidia-smi -L

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
import torch

from datasets import Dataset, load_metric

from google.colab import drive
drive.mount('/content/drive')

# set up the model with Lora

Code reference for setting up Lora: https://colab.research.google.com/drive/14xo6sj4dARk8lXZbOifHEn1f_70qNAwy?usp=sharing#scrollTo=4iwHGzKBN6wk

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb

model_name = "allenai/led-large-16384"
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    load_in_8bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="Seq2Seq" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 461374464 || trainable%: 0.34090833427660183


In [ ]:
!nvidia-smi

Sun Jul 14 23:01:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              50W / 400W |   1633MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Load data

In [ ]:
# Paths to the files
path_train_long = '/content/drive/MyDrive/W266_Project/Data/long_dialogue_NER_cleaning/train_long_bio_NER.csv'
path_test_long = '/content/drive/MyDrive/W266_Project/Data/long_dialogue_NER_cleaning/test_long_bio_NER.csv'
path_val_long = '/content/drive/MyDrive/W266_Project/Data/long_dialogue_NER_cleaning/val_long_bio_NER.csv'

# Load the datasets
train_long = pd.read_csv(path_train_long)
test_long = pd.read_csv(path_test_long)
val_long = pd.read_csv(path_val_long)

# Check the first few rows to ensure they're loaded correctly
print("long_dialogue_train_data:")
print(train_long.head())

# Check the shape
print("Shape of long_dialogue_train_data:")
print(train_long.shape)
print("Shape of long_dialogue_test_data:")
print(test_long.shape)
print("Shape of long_dialogue_val_data:")
print(val_long.shape)


long_dialogue_train_data:
                                            dialogue  \
0  doctor donna torres , date of birth , 08/01/19...   
1  doctor: Good morning, Mr. patient. I'm Dr. doc...   
2  doctor: Hello Mrs. patient, thank you for comi...   
3  doctor hi virginia how're you today patient i'...   
4  doctor: Hello, Mrs. patient, welcome back. How...   

                                                note  \
0  SUBJECTIVE CHIEF COMPLAINT Annual health maint...   
1  SUBJECTIVE CHIEF COMPLAINT Patient reports fru...   
2  SUBJECTIVE CHIEF COMPLAINT Left arm pain after...   
3  SUBJECTIVE CHIEF COMPLAINT Right knee pain. HI...   
4  SUBJECTIVE CHIEF COMPLAINT Recurrent low back ...   

                                       bio_ner_label  \
0  [{'token': 'anxiety', 'label': 'DISEASE'}, {'t...   
1  [{'token': 'anxiety', 'label': 'DISEASE'}, {'t...   
2  [{'token': 'left arm pain', 'label': 'DISEASE'...   
3  [{'token': 'diabetic', 'label': 'DISEASE'}, {'...   
4  [{'token': 'lower

In [ ]:
train_long.head(1)

,dialogue,note,bio_ner_label,bio_ner_unique_label,bio_ner_no_label,bio_ner_unique_no_label
0,"doctor donna torres , date of birth , 08/01/19...",SUBJECTIVE CHIEF COMPLAINT Annual health maint...,"[{'token': 'anxiety', 'label': 'DISEASE'}, {'t...","[{'token': 'anxiety', 'label': 'DISEASE'}, {'t...","['anxiety', 'anxiety', 'irritability', 'anxiet...","['anxiety', 'irritability', 'progesterone']"


In [ ]:
def convert_to_string(list_str):
    # Convert string representation of list to an actual list
    actual_list = eval(list_str)
    # Join the list elements into a single string separated by ", "
    return ", ".join(actual_list)

# Apply the function to convert each list to a string
for df in [train_long, test_long, val_long]:
    df['bio_ner_no_label'] = df['bio_ner_no_label'].apply(convert_to_string)
    df['bio_ner_unique_no_label'] = df['bio_ner_unique_no_label'].apply(convert_to_string)


In [ ]:
train_long.head(1)

,dialogue,note,bio_ner_label,bio_ner_unique_label,bio_ner_no_label,bio_ner_unique_no_label
0,"doctor donna torres , date of birth , 08/01/19...",SUBJECTIVE CHIEF COMPLAINT Annual health maint...,"[{'token': 'anxiety', 'label': 'DISEASE'}, {'t...","[{'token': 'anxiety', 'label': 'DISEASE'}, {'t...","anxiety, anxiety, irritability, anxiety, anxie...","anxiety, irritability, progesterone"


The model can take 16384 tokens, based on data analysis:
- Min dialogue length: 135
- Ave dialogue length: 426
- Max dialogue length: 3050
- 95% dialogue length: 966
- median dialogue length: 353
- Min summary length: 83
- Ave summary length: 276
- Max summary length: 752
- 95% summary length: 458
- Median summary length: 257

In [ ]:
train_long = Dataset.from_pandas(train_long)
test_long = Dataset.from_pandas(test_long)
val_long = Dataset.from_pandas(val_long)


# Fine-tuning the model

## Tokenization

Code reference for tokenization: https://colab.research.google.com/drive/12LjJazBl7Gam0XBPy_y0CTOJZeZ34c2v?usp=sharing


In [ ]:
# tokennize
max_input_length = 2048
max_output_length = 1024
def process_data_to_model_inputs(batch):
    start_prompt = (
    "Summarize the following patient-doctor dialogue into 2 sections: "
    "\"SUBJECTIVE CHIEF COMPLAINT\" and \"ASSESSMENT AND PLAN\". "
    "Include all medically relevant information, including family history, "
    "diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. "
    "Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. "
    "Entities:\n\n")

    # Define the end of the prompt to indicate where the summary should go
    end_prompt = "\n\nSummary: "

    # Construct the full prompt for each dialogue in the batch, using the corresponding entities
    prompt = [start_prompt + entities + 'Dialogue: \n\n' + dialogue + end_prompt
               for entities, dialogue in zip(batch['bio_ner_unique_no_label'], batch['dialogue'])]
    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["note"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
# map the function
batch_size = 2
train_long = train_long.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","bio_ner_label","bio_ner_unique_label","bio_ner_no_label","bio_ner_unique_no_label"],
)

Map:   0%|          | 0/1102 [00:00<?, ? examples/s]

In [ ]:
test_long = test_long.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","bio_ner_label","bio_ner_unique_label","bio_ner_no_label","bio_ner_unique_no_label"],
)

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

In [ ]:
val_long = val_long.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","bio_ner_label","bio_ner_unique_label","bio_ner_no_label","bio_ner_unique_no_label"],
)

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [ ]:
from random import sample
# check the shape after tokenization
sample_input_id = test_long['input_ids'][0:1]
sample_label = test_long['labels'][0:1]
sample_attention_mask = test_long['attention_mask'][0:1]
sample_global_attention_mask = test_long['global_attention_mask'][0:1]

print("tokenized_train_exp: ", test_long)
print("\nsample_input_id: ", len(sample_input_id[0]), sample_input_id)
print("\nsample_label: ", len(sample_label[0]), sample_label)
print("\nsample_attention_mask: ", len(sample_attention_mask[0]), sample_attention_mask)
print("\nsample_global_attention_mask: ", len(sample_global_attention_mask[0]), sample_global_attention_mask)


tokenized_train_exp:  Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 180
})

sample_input_id:  2048 [[0, 38182, 3916, 2072, 5, 511, 3186, 44914, 6054, 88, 132, 26299, 4544, 36856, 104, 12027, 33302, 10002, 3858, 7720, 597, 15823, 43362, 17831, 113, 8, 22, 17042, 12147, 12613, 4248, 37330, 845, 39682, 70, 25749, 4249, 335, 6, 217, 284, 750, 6, 9726, 6, 375, 1131, 8, 15535, 750, 6, 13998, 18391, 6, 6348, 775, 8, 684, 26331, 4, 8630, 511, 8524, 3175, 8, 4747, 1177, 2192, 27380, 31, 6054, 7, 244, 39186, 1938, 6, 53, 109, 45, 14860, 304, 106, 4, 9860, 2192, 4832, 50118, 50118, 4352, 11341, 6, 12867, 14599, 6, 21768, 6, 11696, 6, 19313, 10100, 433, 6, 7910, 6, 1576, 872, 6, 2329, 225, 12572, 8944, 2911, 19036, 6, 24215, 12572, 8944, 2911, 19036, 23600, 6, 28657, 6, 10759, 33412, 6, 26331, 6, 18881, 6, 34729, 935, 1970, 2199, 6, 28349, 4289, 90, 31470, 2400, 6, 38513, 14057, 6, 4266, 27607, 14057, 6, 9308, 6, 765, 1825, 9, 8016, 6, 70

## Training

In [ ]:

import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from transformers import DataCollatorForSeq2Seq

# ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.optim import AdamW

# Define custom optimizer
optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

# Define the output directory
#repository_id = "./output_directory"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=6,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    #logging & evaluation strategies
    #logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_long,
    eval_dataset=val_long,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),  # Pass the custom optimizer
)

In [ ]:
# Start training
with torch.cuda.amp.autocast():
    trainer.train()


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.578472,0.000000,0.000000,0.000000,0.000000,20.000000
2,No log,2.297104,0.819600,0.000000,0.819400,0.819900,20.000000
3,No log,2.174939,0.000000,0.000000,0.000000,0.000000,20.000000
4,2.713600,2.131502,0.000000,0.000000,0.000000,0.000000,20.000000
5,2.713600,2.125468,0.000000,0.000000,0.000000,0.000000,20.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1253: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1253: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 

In [ ]:
!nvidia-smi

Sun Jul 14 23:43:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              68W / 400W |  13347MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# save the model
trainer.save_model("./led_large_dialogue_G_ACI_bio_unique_ner")
tokenizer.save_pretrained("./led_large_dialogue_G_ACI_bio_unique_ner")

('./led_large_dialogue_G_ACI_bio_unique_ner_1/tokenizer_config.json',
 './led_large_dialogue_G_ACI_bio_unique_ner_1/special_tokens_map.json',
 './led_large_dialogue_G_ACI_bio_unique_ner_1/vocab.json',
 './led_large_dialogue_G_ACI_bio_unique_ner_1/merges.txt',
 './led_large_dialogue_G_ACI_bio_unique_ner_1/added_tokens.json',
 './led_large_dialogue_G_ACI_bio_unique_ner_1/tokenizer.json')

In [ ]:
# push to huggingface
from huggingface_hub import HfApi, HfFolder

model_name = "litt5/led_large_dialogue_G_ACI_bio_unique_ner"
api = HfApi()

# Create the repository if it doesn't exist
api.create_repo(repo_id=model_name, repo_type="model", exist_ok=True)

# Upload the model files
api.upload_folder(
    folder_path="./led_large_dialogue_G_ACI_bio_unique_ner",
    repo_id=model_name,
    repo_type="model"
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/litt5/led_large_dialogue_G_ACI_bio_unique_ner_1/commit/86319bd5e0f991f74a52a08ec5778fce196b1b8a', commit_message='Upload folder using huggingface_hub', commit_description='', oid='86319bd5e0f991f74a52a08ec5778fce196b1b8a', pr_url=None, pr_revision=None, pr_num=None)

# Model evaluation

Code reference for quntination: https://github.com/datasci-w266/2024-summer-main/blob/master/materials/lesson_notebooks/lesson_9_Entities_and_Linking.ipynb

In [ ]:
!pip install -q accelerate
!pip install -q bitsandbytes


In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "litt5/led_large_dialogue_G_ACI_bio_unique_ner"

In [ ]:
from transformers import BitsAndBytesConfig


nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=nf4_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
# Paths to the filespath_train_long
path_test_long = '/content/drive/MyDrive/W266_Project/Data/long_dialogue_NER_cleaning/test_long_bio_NER.csv'

# Load the datasets

test_long = pd.read_csv(path_test_long)


# Check the shape

print("Shape of long_dialogue_test_data:")
print(test_long.shape)



In [ ]:
test_long.head(1)

,dialogue,note,bio_ner_label,bio_ner_unique_label,bio_ner_no_label,bio_ner_unique_no_label
0,"doctor: Good morning, Mr. A. Thank you for com...",SUBJECTIVE CHIEF COMPLAINT New patient evaluat...,"[{'token': 'seizures', 'label': 'DISEASE'}, {'...","[{'token': 'seizures', 'label': 'DISEASE'}, {'...","['seizures', 'hemiparesis', 'C . diff', 'malig...","['seizures', 'hemiparesis', 'C . diff', 'malig..."


In [ ]:
def convert_to_string(list_str):
    # Convert string representation of list to an actual list
    actual_list = eval(list_str)
    # Join the list elements into a single string separated by ", "
    return ", ".join(actual_list)

# Apply the function to convert each list to a string
test_long['bio_ner_no_label'] = test_long['bio_ner_no_label'].apply(convert_to_string)
test_long['bio_ner_unique_no_label'] = test_long['bio_ner_unique_no_label'].apply(convert_to_string)

In [ ]:
test_long.head(1)

,dialogue,note,bio_ner_label,bio_ner_unique_label,bio_ner_no_label,bio_ner_unique_no_label
0,"doctor: Good morning, Mr. A. Thank you for com...",SUBJECTIVE CHIEF COMPLAINT New patient evaluat...,"[{'token': 'seizures', 'label': 'DISEASE'}, {'...","[{'token': 'seizures', 'label': 'DISEASE'}, {'...","seizures, hemiparesis, C . diff, malignant men...","seizures, hemiparesis, C . diff, malignant men..."


In [ ]:
num_chunks = 10

# Split the DataFrame
dfs = np.array_split(test_long, num_chunks)

In [ ]:
# save dialogue and note to list for future Rough score
dialogue_lists = [subset["dialogue"].tolist() for subset in dfs]
note_lists = [subset["note"].tolist() for subset in dfs]

In [ ]:
datasets = []
for i in range(len(dfs)):
    dataset = Dataset.from_pandas(dfs[i])
    datasets.append(dataset)

In [ ]:
datasets

[Dataset({
     features: ['dialogue', 'note', 'bio_ner_label', 'bio_ner_unique_label', 'bio_ner_no_label', 'bio_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'bio_ner_label', 'bio_ner_unique_label', 'bio_ner_no_label', 'bio_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'bio_ner_label', 'bio_ner_unique_label', 'bio_ner_no_label', 'bio_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'bio_ner_label', 'bio_ner_unique_label', 'bio_ner_no_label', 'bio_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'bio_ner_label', 'bio_ner_unique_label', 'bio_ner_no_label', 'bio_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'bio_ner_label', 'bio_ner_unique_label', 'bio_ner_no_label', 'bio_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note',

## Tokenization

In [ ]:
# tokennize
max_input_length = 2048
max_output_length = 1024
def process_data_to_model_inputs(batch):
    start_prompt = (
    "Summarize the following patient-doctor dialogue into 2 sections: "
    "\"SUBJECTIVE CHIEF COMPLAINT\" and \"ASSESSMENT AND PLAN\". "
    "Include all medically relevant information, including family history, "
    "diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. "
    "Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. "
    "Entities:\n\n")

    # Define the end of the prompt to indicate where the summary should go
    end_prompt = "\n\nSummary: "

    # Construct the full prompt for each dialogue in the batch, using the corresponding entities
    prompt = [start_prompt + entities + 'Dialogue: \n\n' + dialogue + end_prompt
               for entities, dialogue in zip(batch['bio_ner_unique_no_label'], batch['dialogue'])]
    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["note"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
batch_size = 2
for i in range(10):
    processed_dataset = datasets[i].map(
        process_data_to_model_inputs,
        batched=True,
        batch_size=batch_size,
        remove_columns=["note", "dialogue","bio_ner_label","bio_ner_unique_label","bio_ner_no_label","bio_ner_unique_no_label"],
    )
    # Create a variable name dynamically
    globals()[f'dataset_{i}'] = processed_dataset

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [ ]:
dataset_0

Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 18
})

In [ ]:
## check the tokenization
sample_input_id = dataset_0['input_ids'][0:1]

In [ ]:
# check the prompt:
decoded_text = tokenizer.decode(sample_input_id[0], skip_special_tokens=True)
decoded_text

'Summarize the following patient-doctor dialogue into 2 sections: "SUBJECTIVE CHIEF COMPLAINT" and "ASSESSMENT AND PLAN". Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. Entities:\n\nseizures, hemiparesis, C. diff, malignant meningioma, brain edema, tumor, swelling, confusion, Keppra, Decadron, oxycodone, aspirin, Prilosec, Dilantin, FlagylDialogue: \n\ndoctor: Good morning, Mr. A. Thank you for coming in today. I understand you\'re here for a new patient evaluation. patient: That\'s right. doctor: Can you tell me a little bit about your medical history? patient: Well, I\'ve had two surgeries in the past. The first was in July of 2006, and the second was in August of 2007. After that, I started having seizures and hemiparesis. I\'ve been in and o

## Prediction

In [ ]:
def model_generation(model, tokenizer, dataset):
    predictions = []

    for example in dataset:
        input_ids = example["input_ids"]
        attention_mask = example["attention_mask"]
        global_attention_mask = example["global_attention_mask"]
        reference_ids = example["labels"]

        # Decode input_ids
        input_text = tokenizer.decode(input_ids, skip_special_tokens=True)

        # Generate summary
        outputs = model.generate(input_ids=torch.tensor([input_ids]).to(device),
                                 attention_mask=torch.tensor([attention_mask]).to(device),
                                 global_attention_mask=torch.tensor([global_attention_mask]).to(device),
                                 num_beams= 4,
                                 min_length=100,
                                 max_length=512,
                                 length_penalty=2,
                                 no_repeat_ngram_size=3)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append to lists for ROUGE calculation
        predictions.append(generated_text)
    return predictions

In [ ]:
with torch.cuda.amp.autocast():
  predictions_sub_0 = model_generation(model, tokenizer, dataset_0)

In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(predictions_sub_0, note_lists[0]):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

In [ ]:
avg_rouge1

0.48782596168454506

In [ ]:
avg_rouge2

0.24553842165583614

In [ ]:
avg_rougeL

0.24091656445790433

In [ ]:
data = {
    "Dialogue": dialogue_lists[0],
    "Reference": note_lists[0],
    "Prediction": predictions_sub_0,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/W266_Project/Data/Results'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"led_large_lora_bio_unique_predictions_sub_0.csv"), index=False)

In [ ]:
with torch.cuda.amp.autocast():
  predictions_sub_1 = model_generation(model, tokenizer, dataset_1)

In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(predictions_sub_1, note_lists[1]):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

In [ ]:
avg_rouge1

0.47619603550043593

In [ ]:
avg_rouge2

0.22726229367338802

In [ ]:
avg_rougeL

0.23078045556507096

In [ ]:
data = {
    "Dialogue": dialogue_lists[1],
    "Reference": note_lists[1],
    "Prediction": predictions_sub_1,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/W266_Project/Data/Results'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"led_large_lora_bio_unique_predictions_sub_1.csv"), index=False)

In [ ]:
with torch.cuda.amp.autocast():
  predictions_sub_2 = model_generation(model, tokenizer, dataset_2)

In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(predictions_sub_2, note_lists[2]):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

In [ ]:
data = {
    "Dialogue": dialogue_lists[2],
    "Reference": note_lists[2],
    "Prediction": predictions_sub_2,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/W266_Project/Data/Results'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"led_large_lora_bio_unique_predictions_sub_2.csv"), index=False)